In [2]:
#import necessary dependencies
import pandas as pd
import numpy as np
import os
import boto3
from botocore.exceptions import NoCredentialsError
import io
from io import StringIO
from dotenv import load_dotenv
import traceback
import sys
from sqlalchemy import create_engine
import configparser
from langdetect import detect
import langdetect.lang_detect_exception
from deep_translator import GoogleTranslator
import re
import numpy as np
import random

In [9]:
%store -r

In [5]:
%store raw_dubai_data
%store raw_la_data
%store raw_london_data
%store raw_london_data
%store raw_miami_data
%store raw_nyc_data
%store raw_sanfransisco_data
%store raw_sydney_data
%store raw_tokyo_data
%store raw_toronto_data
%store raw_data

Stored 'raw_dubai_data' (DataFrame)
Stored 'raw_la_data' (DataFrame)
Stored 'raw_london_data' (DataFrame)
Stored 'raw_london_data' (DataFrame)
Stored 'raw_miami_data' (DataFrame)
Stored 'raw_nyc_data' (DataFrame)
Stored 'raw_sanfransisco_data' (DataFrame)
Stored 'raw_sydney_data' (DataFrame)
Stored 'raw_tokyo_data' (DataFrame)
Stored 'raw_toronto_data' (DataFrame)
Stored 'raw_data' (dict)


In [66]:
%store unique_columns

Stored 'unique_columns' (set)


In [ ]:
%load_ext sql

In [ ]:
config = configparser.ConfigParser()
config.read('.env')

In [ ]:
aws_secret_key = config['AWS']['secret_key']
aws_access_key = config['AWS']['access_key']





In [3]:
#upload raw_data
raw_dubai_data = pd.read_csv("airbnb_listings/DubaiData.csv")
raw_la_data = pd.read_csv("airbnb_listings/LAData.csv")
raw_london_data = pd.read_csv("airbnb_listings/LondonData.csv")
raw_miami_data = pd.read_csv("airbnb_listings/MiamiData.csv")
raw_nyc_data = pd.read_csv("airbnb_listings/NYCData.csv")
raw_sanfransisco_data = pd.read_csv("airbnb_listings/SanFransiscoData.csv")
raw_sydney_data = pd.read_csv("airbnb_listings/SydneyData.csv")
raw_tokyo_data = pd.read_csv("airbnb_listings/TokyoData.csv")
raw_toronto_data = pd.read_csv("airbnb_listings/TorontoData.csv")

In [28]:
raw_data = {
    "DubaiData": raw_dubai_data,
    "LosAngelesData": raw_la_data,
    "LondonData": raw_london_data,
    "MiamiData": raw_miami_data,
    "NewYorkCityData": raw_nyc_data,
    "SanFranciscoData": raw_sanfransisco_data,
    "SydneyData": raw_sydney_data,
    "TokyoData": raw_tokyo_data,
    "TorontoData": raw_toronto_data
}

In [ ]:
#upload to S3_Bucket
s3_client = boto3.client('s3',
                         aws_access_key_id = aws_access_key,
                         aws_secret_access_key =aws_secret_key
                        )

In [ ]:
response = s3_client.create_bucket(
            Bucket = 'raw_airbnb_listings_data',
            CreateBucketConfiguration = {
                'LocationConstraint': 'eu-west-2',
            },
            )
print(response)

In [ ]:
#upload to S3 bucket
def upload_to_s3(df, Bucket, object_key):
    try:
        buffer = io.BytesIO()
        df.to_csv(buffer, index = False)
        buffer.seek(0)
        s3_client.upload_fileobj(buffer, Bucket, object_key)
        print(f"file uploaded to S3://{Bucket}/{object_key}.csv")
    except NoCredentialsError:
        print("Credentials not available or incorrect")
    except Exception as e:
        print(f"Upload to S3://{Bucket}/{object_key} failed: {e}")
        

In [ ]:
#upload raw data

In [10]:
raw_dubai_data.head()

,Listing Title,Property Type,Listing Type,Created Date,Last Scraped Date,Country,City,Zipcode,Currency Native,Number of Reviews,...,Count Available Days LTM,Count Blocked Days LTM,Count Reservation Days LTM,Occupancy Rate LTM,Number of Bookings LTM,Number of Bookings LTM - Number of observed month,Average Daily Rate (USD),Average Daily Rate (Native),Annual Revenue LTM (USD),Annual Revenue LTM (Native)
0,Spacious & Private Townhome | 5 mins drive to ...,Entire rental unit,entire_home,2023-07-31,2024-01-10,AE,Dubai,25314,AED,1,...,28,337,18,77,9,6,145,532.1,3126,11472.4
1,Spacious and Ornate 2BR Apartment in Dubai South,Entire rental unit,entire_home,2023-07-31,2024-01-10,AE,Dubai,25314,AED,24,...,333,32,217,78,24,21,79,289.9,21815,80061.1
2,Exceptional 2BR near Expo City Dubai!,Entire rental unit,entire_home,2023-07-31,2024-01-10,AE,دبي,25314,AED,13,...,219,146,176,96,14,11,70,256.9,14717,54011.4
3,Stylish Apartment Surrounded by Lush Green Parks!,Entire rental unit,entire_home,2023-07-31,2024-01-10,AE,Dubai,25314,AED,12,...,253,112,182,86,9,6,192,704.6,41775,153314.2
4,OFFER! 50% OFF! I am very cool and sleep 6!,Entire condo,entire_home,2023-07-31,2024-01-03,AE,Dubai,25314,AED,10,...,137,228,50,44,6,3,74,271.6,4461,16371.9


In [46]:
raw_la_data.head()

,Listing Title,Property Type,Listing Type,Created Date,Last Scraped Date,Country,State,City,Zipcode,Currency Native,...,Count Available Days LTM,Count Blocked Days LTM,Count Reservation Days LTM,Occupancy Rate LTM,Number of Bookings LTM,Number of Bookings LTM - Number of observed month,Average Daily Rate (USD),Average Daily Rate (Native),Annual Revenue LTM (USD),Annual Revenue LTM (Native)
0,海景泳池休闲别墅,Entire villa,entire_home,2023-07-31,2023-11-01,US,California,Los Angeles,90732,USD,...,149,216,94,76,22,19,976,976,110522,110522
1,Fall Getaway @ 75% off Local Resort Price,Entire home,entire_home,2022-12-12,2023-01-04,US,California,Los Angeles,90732,USD,...,36,329,6,29,1,0,853,853,25485,25485
2,The Peninsula Beach House,Entire home,entire_home,2023-07-31,2024-01-10,US,California,Los Angeles,90732,USD,...,350,15,170,58,19,16,817,817,170126,170126
3,Peninsula ocean View Pool Villa,Entire villa,entire_home,2023-07-31,2023-11-01,US,California,Los Angeles,90732,USD,...,59,306,32,65,1,-2,834,834,31984,31984
4,无敌海景、泳池、烧烤欢乐别墅,Private room in villa,private_room,2023-07-31,2023-11-01,US,California,Los Angeles,90732,USD,...,172,193,57,40,39,36,652,652,44858,44858


In [38]:
raw_london_data.head()

,Listing Title,Property Type,Listing Type,Created Date,Last Scraped Date,Country,State,City,Zipcode,Currency Native,...,Count Available Days LTM,Count Blocked Days LTM,Count Reservation Days LTM,Occupancy Rate LTM,Number of Bookings LTM,Number of Bookings LTM - Number of observed month,Average Daily Rate (USD),Average Daily Rate (Native),Annual Revenue LTM (USD),Annual Revenue LTM (Native)
0,Twin Ensuite with Shower at Colnbrook Lodge Gu...,Private room in guesthouse,private_room,2022-07-23,2022-07-27,GB,UK,Colnbrook,SL3 0,GBP,...,0,365,0,28,0,4,120,94.8,874,690.5
1,Single Shared Bathroom at Colnbrook Lodge Gues...,Private room in guesthouse,private_room,2022-07-23,2022-08-03,GB,UK,Colnbrook,SL3 0,GBP,...,12,353,4,44,0,3,67,52.9,1120,884.8
2,Luxury New Alpine Lodge in Private Walled Garden,Entire cabin,entire_home,2023-01-18,2023-05-03,GB,UK,Claygate,KT10 0,GBP,...,143,222,36,30,8,4,204,161.2,8752,6914.1
3,Chessington World of Adventures House,Entire home,entire_home,2022-07-22,2022-08-03,GB,UK,Greater London,KT9 2,GBP,...,11,354,5,55,0,3,153,120.9,980,774.2
4,Charming house -15min to London,Private room in home,private_room,2022-12-12,2023-10-11,GB,UK,Chessington,KT9 2,GBP,...,365,0,163,54,103,93,46,36.3,9067,7162.9


In [48]:
raw_miami_data.head()

,Listing Title,Property Type,Listing Type,Created Date,Last Scraped Date,Last Host Count Updated Date,Country,State,City,Zipcode,...,Count Available Days LTM,Count Blocked Days LTM,Count Reservation Days LTM,Occupancy Rate LTM,Number of Bookings LTM,Number of Bookings LTM - Number of observed month,Average Daily Rate (USD),Average Daily Rate (Native),Annual Revenue LTM (USD),Annual Revenue LTM (Native)
0,Cozy Family home with a pool,Entire home,entire_home,2023-07-31,2024-01-03,NaN,US,NaN,Miami,33138,...,84,281,6,9,0,-3,222,222,1678,1678
1,Coconut Grove Cottage in the Heart of the Vill...,Entire guesthouse,entire_home,2023-07-31,2023-10-11,NaN,US,Florida,Miami,33133,...,178,187,83,56,21,18,94,94,11470,11470
2,DOWNTOWN MODERN INCREDIBLE VIEWS 2/2 LOFT,Entire rental unit,entire_home,2023-07-31,2024-01-03,NaN,US,Florida,Miami,33132,...,92,273,70,91,15,12,291,291,25863,25863
3,"WELCOME TO PARADISE MIAMI, FLORIDA",Private room in rental unit,private_room,2023-07-31,2024-01-03,NaN,US,Florida,Miami,33129,...,210,155,168,96,37,34,60,60,12096,12096
4,Garden Cottage- Heart of Design District,Entire guesthouse,entire_home,2023-07-31,2024-01-03,NaN,US,Florida,Miami,33137,...,53,312,39,88,2,-1,52,52,2575,2575


In [49]:
raw_nyc_data.head()

,Listing Title,Property Type,Listing Type,Created Date,Last Scraped Date,Country,State,City,Zipcode,Currency Native,...,Count Available Days LTM,Count Blocked Days LTM,Count Reservation Days LTM,Occupancy Rate LTM,Number of Bookings LTM,Number of Bookings LTM - Number of observed month,Average Daily Rate (USD),Average Daily Rate (Native),Annual Revenue LTM (USD),Annual Revenue LTM (Native)
0,Two-Story Condo 4 Houses to the Beach in Holgate,Entire condo,entire_home,2022-12-12,2023-03-16,US,New Jersey,Long Beach Township,8008.0,USD,...,33,332,7,25,0,-3,350,350,2888,2888
1,Spectacular Oceanfront Beach Escape,Entire home,entire_home,2023-07-31,2023-12-13,US,New Jersey,Long Beach,8008.0,USD,...,30,335,8,32,0,-3,2627,2627,25219,25219
2,Adventure Cottage | Near LBI | Million Dollar ...,Entire home,entire_home,2023-07-31,2023-08-16,US,New Jersey,Stafford Township,8092.0,USD,...,106,259,55,60,16,15,232,232,25507,25507
3,Peaceful home,Private room in home,private_room,2023-07-31,2023-12-20,US,New Jersey,Stafford Township,8092.0,USD,...,59,306,32,65,7,4,92,92,3528,3528
4,Serenity over the bridge from LBI.,Private room in house,private_room,2020-07-12,2022-01-19,US,New Jersey,Stafford Township,8050.0,USD,...,0,365,0,60,0,10,140,140,168,168


In [50]:
raw_sanfransisco_data.head()

,Listing Title,Property Type,Listing Type,Created Date,Last Scraped Date,Country,State,City,Zipcode,Currency Native,...,Count Available Days LTM,Count Blocked Days LTM,Count Reservation Days LTM,Occupancy Rate LTM,Number of Bookings LTM,Number of Bookings LTM - Number of observed month,Average Daily Rate (USD),Average Daily Rate (Native),Annual Revenue LTM (USD),Annual Revenue LTM (Native)
0,"Beach view, newly furnished 2 bdr apartment in SF",Entire rental unit,entire_home,2023-07-31,2023-12-13,US,California,San Francisco,94116.0,USD,...,99,266,64,78,1,-2,200,200,15574,15574
1,Ocean Beach gem w/ surf views,Entire rental unit,entire_home,2023-07-31,2024-01-10,US,California,San Francisco,94116.0,USD,...,344,21,267,93,40,37,306,306,103096,103096
2,Ocean Beach sky apartment,Entire rental unit,entire_home,2023-07-31,2023-10-21,US,California,San Francisco,94116.0,USD,...,110,255,101,100,9,6,252,252,27720,27720
3,Bright 2 Bedroom Home Walk To Beach,Entire home,entire_home,2023-07-31,2024-01-10,US,California,San Francisco,94116.0,USD,...,49,316,18,44,1,-2,141,141,3315,3315
4,San Francisco Beach House,Entire home,entire_home,2023-07-31,2024-01-10,US,California,San Francisco,94116.0,USD,...,108,257,67,74,1,-2,131,131,10920,10920


In [51]:
raw_sydney_data.head()

,Listing Title,Property Type,Listing Type,Created Date,Last Scraped Date,Country,State,City,Zipcode,Currency Native,...,Count Available Days LTM,Count Blocked Days LTM,Count Reservation Days LTM,Occupancy Rate LTM,Number of Bookings LTM,Number of Bookings LTM - Number of observed month,Average Daily Rate (USD),Average Daily Rate (Native),Annual Revenue LTM (USD),Annual Revenue LTM (Native)
0,KUI COTTAGE private room with ensuite,Private room in home,private_room,2023-07-31,2024-01-10,AU,New South Wales,Camden,2570.0,AUD,...,61,304,17,33,10,7,78,117.0,1570,2355.0
1,River View Appartment (on Camden bike track),Entire rental unit,entire_home,2023-07-31,2024-01-10,AU,New South Wales,Camden,2570.0,AUD,...,351,14,219,75,34,31,93,139.5,26012,39018.0
2,Modern Luxury with Nature,Entire home,entire_home,2023-07-31,2024-01-10,AU,New South Wales,Spring Farm,2570.0,AUD,...,121,244,86,85,21,18,103,154.5,10594,15891.0
3,Tre Sorelle Camden,Entire rental unit,entire_home,2023-07-31,2024-01-10,AU,New South Wales,Camden,2570.0,AUD,...,287,78,164,69,41,38,180,270.0,35645,53467.5
4,Cozy 50s Cottage in Camden (our fibro Château :),Entire bungalow,entire_home,2023-07-31,2024-01-10,AU,New South Wales,Elderslie,2570.0,AUD,...,131,234,55,50,6,3,129,193.5,8804,13206.0


In [52]:
raw_tokyo_data.head()

,Listing Title,Property Type,Listing Type,Created Date,Last Scraped Date,Country,State,City,Zipcode,Currency Native,...,Count Available Days LTM,Count Blocked Days LTM,Count Reservation Days LTM,Occupancy Rate LTM,Number of Bookings LTM,Number of Bookings LTM - Number of observed month,Average Daily Rate (USD),Average Daily Rate (Native),Annual Revenue LTM (USD),Annual Revenue LTM (Native)
0,Yamato Sta 1-min room A 203 (studio),Private room in rental unit,private_room,2023-07-31,2024-01-10,JP,Tokyo,Yamato shi,NaN,JPY,...,61,304,6,12,6,3,40,5831.6,653,95200.9
1,Yamato Sta 1-min room B 201 (studio),Private room in rental unit,private_room,2023-07-31,2024-01-10,JP,Tokyo,Yamato,NaN,JPY,...,206,159,106,62,12,9,41,5977.4,5957,868471.0
2,Yamato Sta 1-min room C 305 (long stay studio),Private room in rental unit,private_room,2023-07-31,2024-01-10,JP,Tokyo,Yamato-shi,NaN,JPY,...,246,119,74,36,6,3,38,5540.0,3725,543067.8
3,Fujino House / 藤野ハウス,Entire home,entire_home,2022-07-22,2024-01-10,JP,Tokyo,Sagamihara,NaN,JPY,...,286,79,211,89,57,46,64,9330.6,18514,2699156.1
4,【１棟貸し切りコンドミニアム 相模湖BASSHOUSE】,Entire bungalow,entire_home,2023-07-31,2024-01-10,JP,Tokyo,"Midori-ku, Sagamihara",NaN,JPY,...,327,38,193,71,78,75,203,29595.4,49393,7201005.5


In [16]:
raw_toronto_data.head()

,Listing Title,Property Type,Listing Type,Created Date,Last Scraped Date,Country,State,City,Currency Native,Number of Reviews,...,Count Available Days LTM,Count Blocked Days LTM,Count Reservation Days LTM,Occupancy Rate LTM,Number of Bookings LTM,Number of Bookings LTM - Number of observed month,Average Daily Rate (USD),Average Daily Rate (Native),Annual Revenue LTM (USD),Annual Revenue LTM (Native)
0,Waterfront Cozy Escape,Private room in rental unit,private_room,2023-07-31,2024-01-10,CA,Ontario,Toronto,CAD,79,...,224,141,162,87,26,23,100,134.0,21568,28901.1
1,Live by the Lake Ontario-Entire Apartment,Entire condo,entire_home,2023-07-31,2024-01-10,CA,Ontario,Toronto,CAD,19,...,32,333,19,71,2,-1,102,136.7,2407,3225.4
2,Home away from home.,Private room in rental unit,private_room,2023-07-31,2024-01-10,CA,Ontario,Toronto,CAD,5,...,173,192,73,51,3,0,74,99.2,6550,8777.0
3,"❤️Beautiful HOUSE!, near everything! LAKE+WIFI...",Entire home,entire_home,2023-07-31,2024-01-10,CA,Ontario,Toronto,CAD,50,...,78,287,33,51,5,2,65,87.1,3176,4255.8
4,💕BEAUTIFUL BSMT STUDIO By LAKE! + wifi PRKN & ...,Entire home,entire_home,2023-07-31,2024-01-10,CA,Ontario,Toronto,CAD,46,...,271,94,153,68,21,18,84,112.6,17958,24063.7


In [42]:
data = {}
for key, values in raw_data.items():
    data[key] = [values.shape[1], values.shape[0]]
data_summary = pd.DataFrame(data, index = ['columns', 'rows'])
print(data_summary)

         DubaiData  LosAngelesData  LondonData  MiamiData  NewYorkCityData  \
columns         53              54          54         57               54   
rows         13560           12837       43580      10205            40343   

         SanFranciscoData  SydneyData  TokyoData  TorontoData  
columns                54          54         54           53  
rows                 3801        5938       9661         5900  


In [65]:
#compare columns
unique_columns = set(raw_miami_data.columns).intersection(
    set(raw_dubai_data.columns), 
    set(raw_la_data.columns),
    set(raw_london_data.columns),
    set(raw_nyc_data.columns),
    set(raw_sanfransisco_data.columns),
    set(raw_sydney_data.columns),
    set(raw_tokyo_data.columns),
    set(raw_toronto_data.columns)
)
print(f'Number of columns in common: {len(unique_columns)}')

Number of columns in common: 56


In [65]:
print(f"miami_dubai_column_differences: {set(raw_miami_data).difference(set(raw_dubai_data.columns))}")
print(f"miami_toronto_column_differences: {set(raw_miami_data).difference(set(raw_toronto_data.columns))}")
print(f"miami_la_column_differences: {set(raw_miami_data).difference(set(raw_la_data.columns))}")
print(f"miami_london_column_differences: {set(raw_miami_data).difference(set(raw_london_data.columns))}")
print(f"miami_nyc_column_differences: {set(raw_miami_data).difference(set(raw_nyc_data.columns))}")
print(f"miami_sanfransisco_column_differences: {set(raw_miami_data).difference(set(raw_sanfransisco_data.columns))}")
print(f"miami_sydney_column_differences: {set(raw_miami_data).difference(set(raw_sydney_data.columns))}")
print(f"miami_tokyo_column_differences: {set(raw_miami_data).difference(set(raw_tokyo_data.columns))}")

miami_dubai_column_differences: {'Neighbourhood', 'State', 'Last Host Count Updated Date', 'Metropolitan Statistical Area'}
miami_toronto_column_differences: {'Neighbourhood', 'Zipcode', 'Last Host Count Updated Date', 'Metropolitan Statistical Area'}
miami_la_column_differences: {'Neighbourhood', 'Last Host Count Updated Date', 'Metropolitan Statistical Area'}
miami_london_column_differences: {'Neighbourhood', 'Last Host Count Updated Date', 'Metropolitan Statistical Area'}
miami_nyc_column_differences: {'Neighbourhood', 'Last Host Count Updated Date', 'Metropolitan Statistical Area'}
miami_sanfransisco_column_differences: {'Neighbourhood', 'Last Host Count Updated Date', 'Metropolitan Statistical Area'}
miami_sydney_column_differences: {'Neighbourhood', 'Last Host Count Updated Date', 'Metropolitan Statistical Area'}
miami_tokyo_column_differences: {'Neighbourhood', 'Last Host Count Updated Date', 'Metropolitan Statistical Area'}


In [66]:
raw_miami_data['Neighbourhood']

0                              NaN
1                    Coconut Grove
2        Miami Government District
3                         Brickell
4                  Design District
                   ...            
10200                          NaN
10201                          NaN
10202                          NaN
10203                          NaN
10204                          NaN
Name: Neighbourhood, Length: 10205, dtype: object

In [69]:
raw_miami_data['Metropolitan Statistical Area']

0                                            NaN
1        Miami-Fort Lauderdale-Pompano Beach, FL
2        Miami-Fort Lauderdale-Pompano Beach, FL
3        Miami-Fort Lauderdale-Pompano Beach, FL
4        Miami-Fort Lauderdale-Pompano Beach, FL
                          ...                   
10200                                        NaN
10201                                        NaN
10202                                        NaN
10203                                        NaN
10204                                        NaN
Name: Metropolitan Statistical Area, Length: 10205, dtype: object

In [67]:
raw_miami_data['Last Host Count Updated Date']

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
        ... 
10200    NaN
10201    NaN
10202    NaN
10203    NaN
10204    NaN
Name: Last Host Count Updated Date, Length: 10205, dtype: object

In [71]:
raw_miami_data.drop(columns = ['Neighbourhood', 'Last Host Count Updated Date','Metropolitan Statistical Area'])

,Listing Title,Property Type,Listing Type,Created Date,Last Scraped Date,Country,State,City,Zipcode,Currency Native,...,Count Available Days LTM,Count Blocked Days LTM,Count Reservation Days LTM,Occupancy Rate LTM,Number of Bookings LTM,Number of Bookings LTM - Number of observed month,Average Daily Rate (USD),Average Daily Rate (Native),Annual Revenue LTM (USD),Annual Revenue LTM (Native)
0,Cozy Family home with a pool,Entire home,entire_home,2023-07-31,2024-01-03,US,NaN,Miami,33138,USD,...,84,281,6,9,0,-3,222,222,1678,1678
1,Coconut Grove Cottage in the Heart of the Vill...,Entire guesthouse,entire_home,2023-07-31,2023-10-11,US,Florida,Miami,33133,USD,...,178,187,83,56,21,18,94,94,11470,11470
2,DOWNTOWN MODERN INCREDIBLE VIEWS 2/2 LOFT,Entire rental unit,entire_home,2023-07-31,2024-01-03,US,Florida,Miami,33132,USD,...,92,273,70,91,15,12,291,291,25863,25863
3,"WELCOME TO PARADISE MIAMI, FLORIDA",Private room in rental unit,private_room,2023-07-31,2024-01-03,US,Florida,Miami,33129,USD,...,210,155,168,96,37,34,60,60,12096,12096
4,Garden Cottage- Heart of Design District,Entire guesthouse,entire_home,2023-07-31,2024-01-03,US,Florida,Miami,33137,USD,...,53,312,39,88,2,-1,52,52,2575,2575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10200,Charming Home in Buena Vista!,Entire home,entire_home,2023-09-26,2023-12-27,US,NaN,Miami,33137,USD,...,30,335,4,16,1,0,500,500,2400,2400
10201,Wyndwood Bungalow,Entire home,entire_home,2023-09-26,2023-12-27,US,NaN,Miami,33127,USD,...,31,334,24,93,0,-1,90,90,2595,2595
10202,Spectacular - 1B Apt in Downtown,Entire rental unit,entire_home,2023-09-26,2023-12-27,US,NaN,Miami,33132,USD,...,31,334,14,54,0,-1,220,220,3683,3683
10203,Amazing ocean & city views loft @brickell down...,Entire loft,entire_home,2023-09-26,2023-12-27,US,NaN,Miami,33131,USD,...,31,334,17,66,3,2,260,260,5320,5320


In [70]:
#add state column to dubai listing
raw_dubai_data['state'] = 'dubai'

In [68]:
#drop last scraped data
def drop_last_scraped_data(df):
    df = df.drop(columns = ['Last Scraped Date'])
    print(f'{df} Last Scraped date dropped')

In [69]:
for df in raw_data:
    raw_data[df] = drop_last_scraped_data(raw_data[df])

                                           Listing Title  \
0      Spacious & Private Townhome | 5 mins drive to ...   
1       Spacious and Ornate 2BR Apartment in Dubai South   
2                  Exceptional 2BR near Expo City Dubai!   
3      Stylish Apartment Surrounded by Lush Green Parks!   
4            OFFER! 50% OFF! I am very cool and sleep 6!   
...                                                  ...   
13555                        STYLEX HOMES & ACCOMODATION   
13556                     Furnished 3 bedrooms apartment   
13557                                        Comfort Inn   
13558              Affordable place for travelers to UAE   
13559                                      Room in Dubai   

                           Property Type  Listing Type Created Date Country  \
0                     Entire rental unit   entire_home   2023-07-31      AE   
1                     Entire rental unit   entire_home   2023-07-31      AE   
2                     Entire rental unit  

In [80]:
#check na and null values
def detect_missing_values(df):
    df_columns = ['Checkout Time', 'Minimum Stay', 'Max Guests', 'Airbnb Cleanliness Rating', 
    'Listing URL', 'Currency Native', 'Created Date', 'Instantbook Enabled', 
    'Average Daily Rate (USD)', 'Check-in Time', 'Cancellation Policy', 'picture_url', 
    'Annual Revenue LTM (USD)', 'Average Daily Rate (Native)', 'Exact Location', 'Airbnb Communication Rating',
    'Airbnb Property ID', 'Airbnb Accuracy Rating', 'Longitude', 'instant_bookable', 'Airbnb Location Rating', 
    'Amenities', 'Pets Allowed', 'Count Blocked Days LTM', 'Listing Type',
    'Number of Bookings LTM - Number of observed month', 'Latitude', 
    'Cleaning Fee (USD)', 'City', 'Extra People Fee (USD)', 'Overall Rating', 
    'Occupancy Rate LTM', 'Annual Revenue LTM (Native)', 'Host Listing Count', 'Bedrooms', 
    'Number of Reviews', 'Listing Title', 'Number of Bookings LTM', 'License', 'Airbnb Host ID', 
    'Count Available Days LTM', 'Bathrooms', 'Cleaning Fee (Native)', 'Airbnb Checkin Rating', 
    'Airbnb Superhost', 'Count Reservation Days LTM', 'Property Type', 
    'Country', 'Airbnb Value Rating', 'Extra People Fee(Native)', 'guest_controls']

    is_na = {}
    for col in df_columns:
        if df[col].isna().any() == True:
            is_na[col] = [df[col].isna().any(), df[col].isna().sum()]
    print(f'{len(is_na)}\n')
    for key, value in is_na.items():
        print(f'{key}: {value}\n ')
    
    return is_na   

In [81]:
#apply detect values function
print(detect_missing_values(raw_la_data))

13

Checkout Time: [True, 1040]
 
Airbnb Cleanliness Rating: [True, 1776]
 
Check-in Time: [True, 460]
 
Cancellation Policy: [True, 4449]
 
Airbnb Communication Rating: [True, 1575]
 
Airbnb Accuracy Rating: [True, 1575]
 
instant_bookable: [True, 4449]
 
Airbnb Location Rating: [True, 1575]
 
Overall Rating: [True, 1576]
 
Host Listing Count: [True, 4122]
 
License: [True, 7456]
 
Airbnb Checkin Rating: [True, 1575]
 
Airbnb Value Rating: [True, 1576]
 
{'Checkout Time': [True, 1040], 'Airbnb Cleanliness Rating': [True, 1776], 'Check-in Time': [True, 460], 'Cancellation Policy': [True, 4449], 'Airbnb Communication Rating': [True, 1575], 'Airbnb Accuracy Rating': [True, 1575], 'instant_bookable': [True, 4449], 'Airbnb Location Rating': [True, 1575], 'Overall Rating': [True, 1576], 'Host Listing Count': [True, 4122], 'License': [True, 7456], 'Airbnb Checkin Rating': [True, 1575], 'Airbnb Value Rating': [True, 1576]}


In [82]:
print(detect_missing_values(raw_dubai_data))

13

Checkout Time: [True, 1869]
 
Airbnb Cleanliness Rating: [True, 3596]
 
Check-in Time: [True, 644]
 
Cancellation Policy: [True, 9161]
 
Airbnb Communication Rating: [True, 3592]
 
Airbnb Accuracy Rating: [True, 3592]
 
instant_bookable: [True, 9161]
 
Airbnb Location Rating: [True, 3592]
 
Overall Rating: [True, 3614]
 
Host Listing Count: [True, 8756]
 
License: [True, 11602]
 
Airbnb Checkin Rating: [True, 3592]
 
Airbnb Value Rating: [True, 3592]
 
{'Checkout Time': [True, 1869], 'Airbnb Cleanliness Rating': [True, 3596], 'Check-in Time': [True, 644], 'Cancellation Policy': [True, 9161], 'Airbnb Communication Rating': [True, 3592], 'Airbnb Accuracy Rating': [True, 3592], 'instant_bookable': [True, 9161], 'Airbnb Location Rating': [True, 3592], 'Overall Rating': [True, 3614], 'Host Listing Count': [True, 8756], 'License': [True, 11602], 'Airbnb Checkin Rating': [True, 3592], 'Airbnb Value Rating': [True, 3592]}


In [83]:
print(detect_missing_values(raw_london_data))

14

Checkout Time: [True, 5320]
 
Airbnb Cleanliness Rating: [True, 7385]
 
Check-in Time: [True, 2616]
 
Cancellation Policy: [True, 20839]
 
Airbnb Communication Rating: [True, 6428]
 
Airbnb Accuracy Rating: [True, 6429]
 
instant_bookable: [True, 20838]
 
Airbnb Location Rating: [True, 6429]
 
Overall Rating: [True, 6476]
 
Host Listing Count: [True, 19367]
 
Listing Title: [True, 4]
 
License: [True, 43580]
 
Airbnb Checkin Rating: [True, 6430]
 
Airbnb Value Rating: [True, 6429]
 
{'Checkout Time': [True, 5320], 'Airbnb Cleanliness Rating': [True, 7385], 'Check-in Time': [True, 2616], 'Cancellation Policy': [True, 20839], 'Airbnb Communication Rating': [True, 6428], 'Airbnb Accuracy Rating': [True, 6429], 'instant_bookable': [True, 20838], 'Airbnb Location Rating': [True, 6429], 'Overall Rating': [True, 6476], 'Host Listing Count': [True, 19367], 'Listing Title': [True, 4], 'License': [True, 43580], 'Airbnb Checkin Rating': [True, 6430], 'Airbnb Value Rating': [True, 6429]}


In [84]:
print(detect_missing_values(raw_miami_data))

17

Checkout Time: [True, 802]
 
Airbnb Cleanliness Rating: [True, 1393]
 
Check-in Time: [True, 324]
 
Cancellation Policy: [True, 5745]
 
Airbnb Communication Rating: [True, 1278]
 
Airbnb Accuracy Rating: [True, 1278]
 
instant_bookable: [True, 5745]
 
Airbnb Location Rating: [True, 1278]
 
Cleaning Fee (USD): [True, 5744]
 
Extra People Fee (USD): [True, 5744]
 
Overall Rating: [True, 2523]
 
Host Listing Count: [True, 5339]
 
License: [True, 10117]
 
Cleaning Fee (Native): [True, 5744]
 
Airbnb Checkin Rating: [True, 1278]
 
Airbnb Value Rating: [True, 1278]
 
Extra People Fee(Native): [True, 5744]
 
{'Checkout Time': [True, 802], 'Airbnb Cleanliness Rating': [True, 1393], 'Check-in Time': [True, 324], 'Cancellation Policy': [True, 5745], 'Airbnb Communication Rating': [True, 1278], 'Airbnb Accuracy Rating': [True, 1278], 'instant_bookable': [True, 5745], 'Airbnb Location Rating': [True, 1278], 'Cleaning Fee (USD)': [True, 5744], 'Extra People Fee (USD)': [True, 5744], 'Overall Ra

In [89]:
print(detect_missing_values(raw_nyc_data))

13

Checkout Time: [True, 3952]
 
Airbnb Cleanliness Rating: [True, 4833]
 
Check-in Time: [True, 1922]
 
Cancellation Policy: [True, 17276]
 
Airbnb Communication Rating: [True, 4243]
 
Airbnb Accuracy Rating: [True, 4243]
 
instant_bookable: [True, 17274]
 
Airbnb Location Rating: [True, 4243]
 
Overall Rating: [True, 4281]
 
Host Listing Count: [True, 15766]
 
License: [True, 40222]
 
Airbnb Checkin Rating: [True, 4243]
 
Airbnb Value Rating: [True, 4244]
 
{'Checkout Time': [True, 3952], 'Airbnb Cleanliness Rating': [True, 4833], 'Check-in Time': [True, 1922], 'Cancellation Policy': [True, 17276], 'Airbnb Communication Rating': [True, 4243], 'Airbnb Accuracy Rating': [True, 4243], 'instant_bookable': [True, 17274], 'Airbnb Location Rating': [True, 4243], 'Overall Rating': [True, 4281], 'Host Listing Count': [True, 15766], 'License': [True, 40222], 'Airbnb Checkin Rating': [True, 4243], 'Airbnb Value Rating': [True, 4244]}


In [88]:
print(detect_missing_values(raw_sanfransisco_data))

13

Checkout Time: [True, 342]
 
Airbnb Cleanliness Rating: [True, 271]
 
Check-in Time: [True, 149]
 
Cancellation Policy: [True, 687]
 
Airbnb Communication Rating: [True, 254]
 
Airbnb Accuracy Rating: [True, 254]
 
instant_bookable: [True, 687]
 
Airbnb Location Rating: [True, 255]
 
Overall Rating: [True, 251]
 
Host Listing Count: [True, 620]
 
License: [True, 1183]
 
Airbnb Checkin Rating: [True, 255]
 
Airbnb Value Rating: [True, 255]
 
{'Checkout Time': [True, 342], 'Airbnb Cleanliness Rating': [True, 271], 'Check-in Time': [True, 149], 'Cancellation Policy': [True, 687], 'Airbnb Communication Rating': [True, 254], 'Airbnb Accuracy Rating': [True, 254], 'instant_bookable': [True, 687], 'Airbnb Location Rating': [True, 255], 'Overall Rating': [True, 251], 'Host Listing Count': [True, 620], 'License': [True, 1183], 'Airbnb Checkin Rating': [True, 255], 'Airbnb Value Rating': [True, 255]}


In [87]:
print(detect_missing_values(raw_sydney_data))

14

Checkout Time: [True, 386]
 
Airbnb Cleanliness Rating: [True, 582]
 
Check-in Time: [True, 218]
 
Cancellation Policy: [True, 910]
 
Airbnb Communication Rating: [True, 539]
 
Airbnb Accuracy Rating: [True, 539]
 
instant_bookable: [True, 910]
 
Airbnb Location Rating: [True, 539]
 
Overall Rating: [True, 540]
 
Host Listing Count: [True, 885]
 
Listing Title: [True, 1]
 
License: [True, 615]
 
Airbnb Checkin Rating: [True, 539]
 
Airbnb Value Rating: [True, 539]
 
{'Checkout Time': [True, 386], 'Airbnb Cleanliness Rating': [True, 582], 'Check-in Time': [True, 218], 'Cancellation Policy': [True, 910], 'Airbnb Communication Rating': [True, 539], 'Airbnb Accuracy Rating': [True, 539], 'instant_bookable': [True, 910], 'Airbnb Location Rating': [True, 539], 'Overall Rating': [True, 540], 'Host Listing Count': [True, 885], 'Listing Title': [True, 1], 'License': [True, 615], 'Airbnb Checkin Rating': [True, 539], 'Airbnb Value Rating': [True, 539]}


In [86]:
print(detect_missing_values(raw_tokyo_data))

12

Checkout Time: [True, 311]
 
Airbnb Cleanliness Rating: [True, 1204]
 
Check-in Time: [True, 123]
 
Cancellation Policy: [True, 2935]
 
Airbnb Communication Rating: [True, 1029]
 
Airbnb Accuracy Rating: [True, 1029]
 
instant_bookable: [True, 2935]
 
Airbnb Location Rating: [True, 1029]
 
Overall Rating: [True, 1033]
 
Host Listing Count: [True, 2802]
 
Airbnb Checkin Rating: [True, 1029]
 
Airbnb Value Rating: [True, 1029]
 
{'Checkout Time': [True, 311], 'Airbnb Cleanliness Rating': [True, 1204], 'Check-in Time': [True, 123], 'Cancellation Policy': [True, 2935], 'Airbnb Communication Rating': [True, 1029], 'Airbnb Accuracy Rating': [True, 1029], 'instant_bookable': [True, 2935], 'Airbnb Location Rating': [True, 1029], 'Overall Rating': [True, 1033], 'Host Listing Count': [True, 2802], 'Airbnb Checkin Rating': [True, 1029], 'Airbnb Value Rating': [True, 1029]}


In [85]:
print(detect_missing_values(raw_toronto_data))

14

Checkout Time: [True, 471]
 
Airbnb Cleanliness Rating: [True, 606]
 
Check-in Time: [True, 264]
 
Cancellation Policy: [True, 1515]
 
Airbnb Communication Rating: [True, 552]
 
Airbnb Accuracy Rating: [True, 552]
 
instant_bookable: [True, 1515]
 
Airbnb Location Rating: [True, 552]
 
Overall Rating: [True, 557]
 
Host Listing Count: [True, 1444]
 
Listing Title: [True, 1]
 
License: [True, 2437]
 
Airbnb Checkin Rating: [True, 552]
 
Airbnb Value Rating: [True, 552]
 
{'Checkout Time': [True, 471], 'Airbnb Cleanliness Rating': [True, 606], 'Check-in Time': [True, 264], 'Cancellation Policy': [True, 1515], 'Airbnb Communication Rating': [True, 552], 'Airbnb Accuracy Rating': [True, 552], 'instant_bookable': [True, 1515], 'Airbnb Location Rating': [True, 552], 'Overall Rating': [True, 557], 'Host Listing Count': [True, 1444], 'Listing Title': [True, 1], 'License': [True, 2437], 'Airbnb Checkin Rating': [True, 552], 'Airbnb Value Rating': [True, 552]}


In [6]:
#generate dimension tables id
def generate_ids(df, df_acroynm, dim_acroynm):
    size = len(df)
    random_numbers = np.random.randint(low = 10000, high = 90000, size = size)
    ids = [df_acroynm + dim_acroynm + str(number) for number in random_numbers]
    return ids

def add_id_columns(df,  df_acroynm):
    ids = {'listingID':'LIS', 'amenitiesID':'AMT', 'ratingID':'RTE', 'bookingID':'BKD'}
    for id in ids:
       df[id] = generate_ids(df, df_acroynm, ids[id])
       print(f'{df} ids created sucessfully')
  

In [47]:
#add columns
add_id_columns(raw_dubai_data, 'DUB')
add_id_columns(raw_la_data, 'LOC')
add_id_columns(raw_london_data, 'LON')
add_id_columns(raw_miami_data, 'MIA')
add_id_columns(raw_nyc_data, 'NYC')
add_id_columns(raw_sanfransisco_data, 'SAN')
add_id_columns(raw_sydney_data, 'SYD')
add_id_columns(raw_tokyo_data, 'TOK')
add_id_columns(raw_toronto_data, 'TOR')


,Listing Title,Property Type,Listing Type,Created Date,Last Scraped Date,Country,State,City,Currency Native,Number of Reviews,...,Number of Bookings LTM,Number of Bookings LTM - Number of observed month,Average Daily Rate (USD),Average Daily Rate (Native),Annual Revenue LTM (USD),Annual Revenue LTM (Native),listingID,amenitiesID,ratinID,bookingID
0,Waterfront Cozy Escape,Private room in rental unit,private_room,2023-07-31,2024-01-10,CA,Ontario,Toronto,CAD,79,...,26,23,100,134.0,21568,28901.1,TORLIS41088,TORAMT56497,TORRTE68876,TORBKD65263
1,Live by the Lake Ontario-Entire Apartment,Entire condo,entire_home,2023-07-31,2024-01-10,CA,Ontario,Toronto,CAD,19,...,2,-1,102,136.7,2407,3225.4,TORLIS57588,TORAMT23051,TORRTE35884,TORBKD60493
2,Home away from home.,Private room in rental unit,private_room,2023-07-31,2024-01-10,CA,Ontario,Toronto,CAD,5,...,3,0,74,99.2,6550,8777.0,TORLIS73911,TORAMT57113,TORRTE10123,TORBKD34477
3,"❤️Beautiful HOUSE!, near everything! LAKE+WIFI...",Entire home,entire_home,2023-07-31,2024-01-10,CA,Ontario,Toronto,CAD,50,...,5,2,65,87.1,3176,4255.8,TORLIS63777,TORAMT33206,TORRTE55391,TORBKD56375
4,💕BEAUTIFUL BSMT STUDIO By LAKE! + wifi PRKN & ...,Entire home,entire_home,2023-07-31,2024-01-10,CA,Ontario,Toronto,CAD,46,...,21,18,84,112.6,17958,24063.7,TORLIS17780,TORAMT48737,TORRTE39708,TORBKD41780
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5895,3 Bedrooms-3 bathrooms - Entire house to enjoy!,Entire home,entire_home,2023-07-31,2024-01-10,CA,Ontario,Toronto,CAD,6,...,1,-2,51,68.3,243,325.6,TORLIS47484,TORAMT49129,TORRTE88770,TORBKD52840
5896,Bright and Beautiful.,Private room in home,private_room,2023-07-31,2024-01-10,CA,Ontario,Toronto,CAD,11,...,11,8,36,48.2,1709,2290.1,TORLIS73254,TORAMT28866,TORRTE69322,TORBKD40535
5897,Sparkling clean home,Private room in home,private_room,2023-07-31,2024-01-10,CA,Ontario,Toronto,CAD,13,...,1,-2,35,46.9,2891,3873.9,TORLIS80845,TORAMT72564,TORRTE68003,TORBKD63626
5898,Green,Private room in home,private_room,2023-07-31,2024-01-10,CA,Ontario,Toronto,CAD,49,...,1,-2,25,33.5,132,176.9,TORLIS49079,TORAMT42745,TORRTE31408,TORBKD82421


In [ ]:
#remove emojis characters

In [25]:
def translate_row(row):
    translation = GoogleTranslator(source = 'auto', target='en').translate(row)
    return translation

In [26]:
def detect_lang(row):
    try:
        detect(row)
    except langdetect.lang_detect_exception.LangDetectException:
        pass

In [36]:
def replace_non_english(df):
    df_copy = df.copy()
    for col in df_copy:
        for idx, value in enumerate(df_copy[col]):
            if isinstance(value, str) and detect_lang(value) != 'en':
                translation = translate_row(value)
                df_copy.at[idx, col] = translation

                if col == 'Listing Title' and detect_lang(translation) != 'en':
                    
                    for loc in ['City', 'Zipcode', 'State']:
                        if pd.notnull(df_copy.at[idx, loc]) and pd.notnull(df_copy.at[idx, 'Bedrooms']):
                            df_copy.at[idx, 'Listing Title'] = str(df_copy.at[idx, 'Bedrooms']) + ' ' + str(df.at[idx, loc])
                            break
    return df_copy


In [ ]:
def create_dimension_fact_tables(df):
    listing_dim = df[['listingID','Listing Title', 'Property Type', 'Listing Type', 'Listing URL', 'Created Date', 'Currency Native']]
    location_dim = df[['Airbnb Property ID', 'country', 'state', 'city', 'zipcode', 'latitude', 'longtitude', 'Exact Location']]
    host_dim = df[['Airbnb Host ID', 'Host Listing Count']]
    amenities_dim = df[['amenitiesID', 'amenities']]]
    rating_dim = df[['ratingID', 'Overall Rating', 'Airbnb Communication Rating' ,'Airbnb Accuracy Rating'
                     , 'Airbnb Cleanliness Rating', 'Airbnb Checkin Rating', 'Airbnb Location Rating', 'Airbnb Value Rating']]
    booking_dim = df[['Booking ID', 'Check-in Time', 'Checkout Time', 'Minimum Stay']]
    booking_fact = df[['bookingID', 'listingID', 'Airbnb Host ID', 'Airbnb Property ID', 'amenitiesID',
                    'ratingID', 'Number of Reviews', 'Bedrooms', 'Bathrooms', 'Max Guests', 'Airbnb Superhost',
                    'Cancellation Policy', 'Cleaning Fee (USD)', 'Cleaning Fee (Native)', 'Extra People Fee (USD)',
                    'Extra People Fee (Native)', 'instant_bookable', 'Pets Allowed', 'Occupancy Rate LTM',
                    'Number of Bookings LTM', 'Number of Observed Months', 'Average Daily Rate (USD)', 'Annual Revenue LTM (USD)']]
    

In [44]:
print(raw_miami_data[['Zipcode']])

       Zipcode
0        33138
1        33133
2        33132
3        33129
4        33137
...        ...
10200    33137
10201    33127
10202    33132
10203    33131
10204    33127

[10205 rows x 1 columns]


In [60]:
raw_miami_data[['Listing Title', 'Property Type', 'Listing Type', 'Created Date',
       'Last Scraped Date', 'Last Host Count Updated Date', 'Country', 'State',
       'City', 'Zipcode', 'Neighbourhood', 'Metropolitan Statistical Area']]

,Listing Title,Property Type,Listing Type,Created Date,Last Scraped Date,Last Host Count Updated Date,Country,State,City,Zipcode,Neighbourhood,Metropolitan Statistical Area
0,Cozy Family home with a pool,Entire home,entire_home,2023-07-31,2024-01-03,NaN,US,NaN,Miami,33138,NaN,NaN
1,Coconut Grove Cottage in the Heart of the Vill...,Entire guesthouse,entire_home,2023-07-31,2023-10-11,NaN,US,Florida,Miami,33133,Coconut Grove,"Miami-Fort Lauderdale-Pompano Beach, FL"
2,DOWNTOWN MODERN INCREDIBLE VIEWS 2/2 LOFT,Entire rental unit,entire_home,2023-07-31,2024-01-03,NaN,US,Florida,Miami,33132,Miami Government District,"Miami-Fort Lauderdale-Pompano Beach, FL"
3,"WELCOME TO PARADISE MIAMI, FLORIDA",Private room in rental unit,private_room,2023-07-31,2024-01-03,NaN,US,Florida,Miami,33129,Brickell,"Miami-Fort Lauderdale-Pompano Beach, FL"
4,Garden Cottage- Heart of Design District,Entire guesthouse,entire_home,2023-07-31,2024-01-03,NaN,US,Florida,Miami,33137,Design District,"Miami-Fort Lauderdale-Pompano Beach, FL"
...,...,...,...,...,...,...,...,...,...,...,...,...
10200,Charming Home in Buena Vista!,Entire home,entire_home,2023-09-26,2023-12-27,NaN,US,NaN,Miami,33137,NaN,NaN
10201,Wyndwood Bungalow,Entire home,entire_home,2023-09-26,2023-12-27,NaN,US,NaN,Miami,33127,NaN,NaN
10202,Spectacular - 1B Apt in Downtown,Entire rental unit,entire_home,2023-09-26,2023-12-27,NaN,US,NaN,Miami,33132,NaN,NaN
10203,Amazing ocean & city views loft @brickell down...,Entire loft,entire_home,2023-09-26,2023-12-27,NaN,US,NaN,Miami,33131,NaN,NaN


In [57]:
raw_miami_data.columns

Index(['Listing Title', 'Property Type', 'Listing Type', 'Created Date',
       'Last Scraped Date', 'Last Host Count Updated Date', 'Country', 'State',
       'City', 'Zipcode', 'Neighbourhood', 'Metropolitan Statistical Area',
       'Currency Native', 'Number of Reviews', 'Bedrooms', 'Bathrooms',
       'Max Guests', 'Airbnb Superhost', 'Cancellation Policy',
       'Cleaning Fee (USD)', 'Cleaning Fee (Native)', 'Extra People Fee (USD)',
       'Extra People Fee(Native)', 'Check-in Time', 'Checkout Time',
       'Minimum Stay', 'Latitude', 'Longitude', 'Exact Location',
       'Overall Rating', 'Airbnb Communication Rating',
       'Airbnb Accuracy Rating', 'Airbnb Cleanliness Rating',
       'Airbnb Checkin Rating', 'Airbnb Location Rating',
       'Airbnb Value Rating', 'Amenities', 'picture_url', 'License',
       'Airbnb Property ID', 'Airbnb Host ID', 'Host Listing Count',
       'guest_controls', 'instant_bookable', 'Pets Allowed', 'Listing URL',
       'Instantbook Enabled', 

In [70]:
print(unique_columns)

{'Checkout Time', 'Minimum Stay', 'Max Guests', 'Airbnb Cleanliness Rating', 'amenitiesID', 'Listing URL', 'Currency Native', 'Created Date', 'Instantbook Enabled', 'Average Daily Rate (USD)', 'Check-in Time', 'Cancellation Policy', 'picture_url', 'Annual Revenue LTM (USD)', 'Average Daily Rate (Native)', 'Exact Location', 'Airbnb Communication Rating', 'Airbnb Property ID', 'Airbnb Accuracy Rating', 'Longitude', 'instant_bookable', 'Airbnb Location Rating', 'Amenities', 'Pets Allowed', 'Count Blocked Days LTM', 'Listing Type', 'Number of Bookings LTM - Number of observed month', 'ratinID', 'Latitude', 'Cleaning Fee (USD)', 'City', 'Extra People Fee (USD)', 'Overall Rating', 'Occupancy Rate LTM', 'Annual Revenue LTM (Native)', 'Host Listing Count', 'Bedrooms', 'Number of Reviews', 'Listing Title', 'Number of Bookings LTM', 'License', 'Airbnb Host ID', 'Count Available Days LTM', 'Bathrooms', 'Cleaning Fee (Native)', 'Airbnb Checkin Rating', 'listingID', 'Airbnb Superhost', 'Count Reser

In [100]:
raw_miami_data[['Host Listing Count']]

,Host Listing Count
0,NaN
1,1.0
2,3.0
3,3.0
4,2.0
...,...
10200,NaN
10201,NaN
10202,NaN
10203,NaN


In [106]:
raw_miami_data['Currency Native'].isna().any()


False

In [ ]:
  df.colummns.isna.any()
        
        df = df['Listing Title'].replace('', np.NaN).replace('NaN', np.nan)
        df = df['Listing Type'].replace('', np.NaN).replace('NaN', np.nan)
        df = df['Property Type'].replace('', np.NaN).replace('NaN', np.nan)

        
        df = df['Listing URL'].replace('', 'NA').replace('NaN', 'NA')
        df = df['Created Date'].replace('', np.NaN).replace('NaN', np.nan)
        df = df['Created Date'].fillna(method='bfill', inplace=True)
        df = df['Currency Native'].fillna(method='bfill', inplace=True)




        df = df['Minimum Stay'].replace('', np.NaN).replace('NaN', np.nan)
        df = df['Max Guests'].replace('', np.NaN).replace('NaN', np.nan)
        df = df['Airbnb Cleanliness Rating'].replace('', np.NaN).replace('NaN', np.nan)
        df = df['Listing URL'].replace('', 'NA').replace('NaN', 'NA')
        
        df = df['Currency Native'].fillna(method='bfill', inplace=True)
        df = df['Created Date'].fillna(method='bfill', inplace=True)